<div style="color:red;background-color:black">
Diamond Light Source

<h1 style="color:red;background-color:antiquewhite"> Python Language: Interfacing with C/C++</h1>  

©2000-20 Chris Seddon 
</div>

Execute the following cell to activate styling for this tutorial

In [ ]:
from IPython.display import HTML
HTML(f"<style>{open('my.css').read()}</style>")

## Before We Start
In this tutorial we will be using a C/C++ compiler and the "swig" program.  You must run the following "module loads" before running this notebook:
<pre>module load python/3.7
module load gcc/9.2.0
module load swig</pre>  

If you haven't already run these commands, please exit the notebook, run the module loads and in the same command window restart Jupyter notebook.

## Preamble: 1
In this tutorial, we will be showing how to wrap up C and C++ code into a module that can be called as if it were normal Python code; this is achieved by compiling C and C++ code into a shared library.

However, before we start this tutorial, I need to point out that Jupyter notebook doesn't always show the output of Python code.  This is because the notebook server runs code in a subshell and redirects this output of the shell to the browser.  But, some Python code runs in a subshell of a subshell and this output does not get redirected to the browser by the notebook server.

This is particularly true of Python "subprocess" module.  For example is we run code in the following cell, all we see is the reurn code from "subprocess.run()":

In [ ]:
import subprocess
subprocess.run("ls -l".split())

## Preamble: 2
We can circumvent this problem by using pipes.  The "subprocess" module can use pipes to get output from the subshell of the subshell as follows:

In [ ]:
import subprocess
result = subprocess.run("ls -l".split(), stdout=subprocess.PIPE)
print (f"return code: {result.returncode} \nstdout = \n{result.stdout.decode()}")

## Preamble: 3
For the rest of this tutorial, we will wrap up this functionality in a "run" function.  Sometimes, we will also need to specify the directory in which to execute the code:

In [ ]:
import subprocess
def run(cmd, dir="."):
    result = subprocess.run (cmd.split(), stdout=subprocess.PIPE, cwd=dir)
    print (f"{result.stdout.decode()}")
print("run command defined")

## Preamble: 4
Now we are ready to start the tutorial.  Since we are going to explore several examples, we will need several pre-written files.  I've decided to store these files under a "resources" folder for each example.  

We will "cd" to this folder now:

In [ ]:
%cd resources

## Preamble: 5
Now we are ready to start the tutorial.  Since we are going to explore several examples, we will need several pre-written files.  I've decided to store these files under a "resources" folder for each example.  The resources will be split according which example we are using:

In [36]:
run("ls -l", "example1")   # C example
run("ls -l", "example2")   # swig example
run("ls -l", "example3")   # C++ example
run("ls -l", "example4")   # Cython example

total 16
-rw-r--r--  1 seddon  staff  965 23 Mar 14:34 fibmodule.c
-rw-r--r--  1 seddon  staff  770 23 Mar 14:34 setup.py

total 32
drwxr-xr-x  4 seddon  staff  128 24 Mar 15:16 __pycache__
drwxr-xr-x  4 seddon  staff  128 24 Mar 21:23 build
-rw-r--r--  1 seddon  staff  293 24 Mar 15:16 messages.c
-rw-r--r--  1 seddon  staff   98 24 Mar 15:15 messages.h
-rw-r--r--  1 seddon  staff  109 24 Mar 14:32 messages.i
-rw-r--r--  1 seddon  staff  888 23 Mar 20:02 setup.py

total 24
-rw-r--r--  1 seddon  staff   367 24 Mar 16:36 average.hpp
drwxr-xr-x  3 seddon  staff    96 24 Mar 21:23 build
-rw-r--r--@ 1 seddon  staff   244 24 Mar 16:56 myexample.i
-rw-r--r--  1 seddon  staff  1361 24 Mar 20:32 setup.py

total 280
drwxr-xr-x  3 seddon  staff      96 24 Mar 21:03 __pycache__
drwxr-xr-x  4 seddon  staff     128 24 Mar 20:55 build
-rw-r--r--  1 seddon  staff     166 24 Mar 21:23 files.txt
-rw-r--r--  1 seddon  staff  126393 24 Mar 20:55 functions.c
-rw-r--r--  1 seddon  staff     429 24 Mar 20:43

## Example1: 1
When we want to wrap up C and C++ code in a shared library we need to provide:
* the C/C++ code to go in the shared library
* setup.py: instructions on building the library

As an example of C code, we will write a Fibonacci module.  Here is the C code:

In [ ]:
run("cat fibmodule.c", "example1")

## Example1: 2
The C code is in 4 parts:

* the function<pre>int _fib(int n)</pre>
* binding code between C and Python in the function<pre>static PyObject* fib(PyObject* self, PyObject* args)</pre>
* interface definitions<pre>
method table
module definition structure</pre>
* Python entry point to the shared libary<pre>PyMODINIT_FUNC PyInit_fibonacci(void)</pre>  

The `setup.py` file is shown below:

In [ ]:
run("cat setup.py", "example1")

## Example1: 3
The key parts of the above file are:

* creating a "distutils.core.setup" object, containing build details <pre>setup(...)</pre>

* an Extension object defining which C files are to be compiled:
<pre>mymodule = Extension('fibonacci', sources = ['fibmodule.c'])</pre>

* the link between the "setup" object and the "Extension" object
<pre>ext_modules = [mymodule]</pre>

Python has a special way of building the shared library:

In [ ]:
run("python setup.py -v build_ext", "example1")

## Example1: 4
Python creates a shared object file with a ".so" extension in the build directory and some other temporary files.

We now need to install this shared object .  Since we do not have permission to install in Diamond's Anacoda distribution we will have to install locally in our home directory:

In [ ]:
run("python setup.py install --record files.txt --user", "example1")

## Example1: 5
Now we've installed our library, we should clean the build area:

In [ ]:
run("python setup.py clean --all", "example1")
print("staging area cleaned")

## Example1: 6
Time to test our module:

In [ ]:
import fibonacci

print((fibonacci.fib(10)))
print((fibonacci.fib(11)))
print((fibonacci.fib(12)))

## Example1: 7
So our module is working!

Note, we've been keeping a record of the files we've been creating to facilitate the "uninstall":

In [ ]:
run("cat files.txt", "example1")

## Example1: 8
The following code will print out the "egg-info" file:

In [ ]:
installedFiles = open("example1/files.txt", "r")
for file in installedFiles:
    file = file.rstrip()
    if file.endswith("egg-info"):
        f = open(file, 'r')
        print(f.read())

## Example1: 9
Now we can unistall our module:

In [ ]:
def uninstall(example):
    try:
        installedFiles = open(f"{example}/files.txt", "r")
        for file in installedFiles:
            print(f"rm {file}")
            run(f"rm {file}")

        print(f"rm {example}/files.txt")
        run(f"rm {example}/files.txt")
        print("Example uninstalled")
    except:
        print(f"Is {example} already uninstalled?")

uninstall("example1")

## Example2: 1
The above procedure in the same for C++.

The one difficulty with the above is writing the C code `fibmodule.c`.  If you look at the code again, realise that you would expect to have to write the equivalent of the function<pre>int _fib(int n)</pre>
but the binding code requires knowledge of internals of the Python interpretert.  It would be great if we didn't need to write the binding code; that is what the "Swig" library is for.  It also generates the interface tables and entry point.

For more complex examples the "Swig" library saves a lot of work.  We will look at how to modify the above procedure to use "Swig".

We'll be using a different example - a C file that has two messages.  Note that we are using "static" to avoid memory problems; we can't easily allocate memory for a char array in our C code and expect Python to clean it up:

In [ ]:
run("cat messages.c", "example2")

## Example2: 2
Swig needs a special file to tell it what binding code is required:

In [ ]:
run("cat messages.i", "example2")

## Example2: 3
The `setup.py` needs to be changed to reference the "swig" file ("messages.i"):

In [ ]:
run("cat setup.py", "example2")

## Example2: 4
Now we can run `setup.py` to generate binding code etc.  "swig" will be called automatically:

In [ ]:
run("python setup.py -v build_ext", "example2")

## Example2: 5
The binding code produced by "swig" is very verbose.  Here's the first 100 lines ...

In [ ]:
run("head -100 messages_wrap.c", "example2")

## Example2: 6
Next, we can install:

In [ ]:
run("python setup.py install --record files.txt --user", "example2")

## Example2: 7
And test ...

In [ ]:
import mymessages

print(mymessages.say_hello("World"))
print(mymessages.say_goodbye("Universe"))

## Example 2: 8
Finally, clean up

In [ ]:
run("python setup.py clean --all", "example2")
run("rm messages_wrap.c mymessages.py", "example2")
uninstall("example2")

## Example 3: 1
Now for some C++.  

I'm using one C++ header file<pre>average.hpp</pre>

Note that this file uses templates.  "swig" is able to generate compatible Python code to handle this (see later).

In [ ]:
run("cat average.hpp", "example3")

## Example 3.2
The `setup.py` file is shown below.  Note that if this is run on my MacOS machine I have to add extra complile anf link options.  This isn't necessary at Diamond:

In [ ]:
run("cat setup.py", "example3")

## Example 3.3
The `myexample.i` file contains all the information "swig" needs to use the C++ header file.  Note that the file includes a mapping from C++ templates to the equivalent Python lists.

In [ ]:
run("cat myexample.i", "example3")

## Example 3.4
Build and install proceed as before:

In [ ]:
run("python setup.py -v build_ext", "example3")
run("python setup.py install --record files.txt --user", "example3")

## Example 3.5
And now we can test it all works:

In [ ]:
import myexample

iv = myexample.IntVector(4)
dv = myexample.DoubleVector(7)

for i in range(0,4):
    iv[i] = i * 100    
print(myexample.average(iv))

for i in range(0,7):
    dv[i] = float(i * 100)    
print(myexample.average2(dv))

## Example 3.6
Let's clean up:

In [ ]:
run("python setup.py clean --all", "example3")
run("rm myexample.py myexample_wrap.cpp", "example3")
uninstall("example3")

## Example 4.1
Our last example is with Cython.  

Cython is a programming language that is closely related to Python; essentially Python with type declaration for variables.  The idea is to give C-like performance with code that is written mostly in Python with additional C-inspired syntax.

With Cython we write a source file that gets compiled into C code.  The C code then gets build as per our previous examples.

Let's start with the Cython source file.  Note the syntax declaring all variables:

In [ ]:
run("cat functions.pyx", "example4")

## Example 4.2
Let's take a look at `setup.py`:

In [ ]:
run("cat setup.py", "example4")

## Example 4.3
Now we will build and install our Cynthon example:

In [ ]:
run("python setup.py build_ext", "example4")
run("python setup.py install --record files.txt --user", "example4")

## Example4: 4
Let's run the Cython code:

In [ ]:
import functions

functions.say_hello()
functions.say_goodbye()
print((functions.fibonacci(100000)))
print((functions.sumOfSquares(2, 4)))



## Example4: 5
Cython is supposed to be considerably faster than pure Python.  We'll compare timings below.  But for now, here is the equivalent Python code is in `python_functions.py`:

In [ ]:
run("cat python_functions.py", "example4")

## Example4: 6
The following code will perform timings to compare Cython and Python.

In [ ]:
def doTimings():
    t1 = Timings(title = "cython (fibonacci)", setup = "import os, functions",
                                   statement = "functions.fibonacci(100)")
    t2 = Timings(title = "python (fibonacci)", setup = ("import os, sys"            "\n"
                                            "sys.path.append('../src')" "\n"
                                            "import python_functions"   "\n"
                                           ), 
                                   statement = "python_functions.fibonacci(100)")
    u1 = Timings(title = "cython (sumOfSquares)", setup = "import os, functions",
                                   statement = "functions.sumOfSquares(1000, 4000000)")
    u2 = Timings(title = "python (sumOfSquares)", setup = ("import os, sys"            "\n"
                                            "sys.path.append('../src')" "\n"
                                            "import python_functions"   "\n"
                                           ), 
                                   statement = "python_functions.sumOfSquares(1000, 4000000)")
    
    Timings.titles()
    t1.run(10000000)
    t2.run(10000000)
    u1.run(20)
    u2.run(20)


#####################################################
# code to make timings table
import timeit
class Timings(timeit.Timer):
    def __init__(self, title, setup, statement):
        self.title = title
        self.timer = timeit.Timer(stmt = statement, setup = setup)
    def run(self, number):
        t = self.timer.timeit(number=number)
        print(("{:24s}{:10d}{:8.3f}{:8.3f}".format(self.title, number, t, 1/t)))

    def titles():
        print(("{:24s}{:>10s}{:>8s}{:>8s}".format("code", "runs", "time", "1/time")))
        print(("{:24s}{:>10s}{:>8s}{:>8s}".format("====", "====", "====", "======")))


import os
os.chdir("example4")
doTimings()
os.chdir("..")